In [42]:
import os
import math
import numpy as np

from scipy import signal
from scipy.fft import fft, fftshift

import torch
import torch.nn as nn
from torch.nn import functional as F

import cv2
import pickle
from PIL import Image
import matplotlib.pyplot as plt

import IPython

In [43]:
import librosa
import librosa.display as display

In [44]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

CUDA_VISIBLE_DEVICES=7
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

Mon Nov 21 17:46:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:1A:00.0 Off |                  N/A |
| 30%   28C    P8    22W / 250W |  10560MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:1B:00.0 Off |                  N/A |
| 30%   

In [45]:
class LHB_Dataset(torch.utils.data.Dataset):

    def __init__(self, path, ext):
        self.path = path
        self.ext = ext
        self.len = len(os.listdir(self.path))
        self.items_in_dir = os.listdir(self.path)


    def __len__(self):
        return self.len

    
    def __getitem__(self, idx):
       
        name = self.path + '/' + self.items_in_dir[idx] #self.path + '/' + str(idx) + "." + self.ext

        with open(name, 'rb') as fd:
            image = pickle.load(fd)

        return image

In [46]:
train_path = '../UnzippedDataset/train'

train_ds = LHB_Dataset(train_path, 'mus')

print(train_ds[0].shape)
print(len(train_ds))

#train
train_generator = torch.Generator(device='cpu')
train_generator.manual_seed(13)
trainloader = torch.utils.data.DataLoader(
                                            dataset=train_ds, 
                                            batch_size=1, 
                                            shuffle=True,
                                            generator=train_generator
                                        )

(1321967,)
2664


In [47]:
def create_dataset(trainloader):
    
    index = 0
    
    NUM_COLS = 1024
    
    for data in trainloader: 
        print("WORKING ON SPECTROGRAM NUMBER ", index)
        data = data.squeeze(dim=0) # one song 

        # Compute spectrograms
        train_stft = librosa.stft(np.asarray(data), n_fft=4096, win_length=4096, window=signal.windows.hamming(4096))
        train_spectrogram = torch.tensor(librosa.amplitude_to_db(abs(train_stft)))

        rows = train_spectrogram.shape[0]
        cols = train_spectrogram.shape[1]

        if cols < NUM_COLS:
            continue

        if cols % NUM_COLS > 0:
            cols_to_add = NUM_COLS - cols % NUM_COLS
            new_data = torch.zeros(size = (rows, cols + cols_to_add))
            new_data[:, : cols] = train_spectrogram
            new_data[:, cols : cols+cols_to_add] = train_spectrogram[:, -cols_to_add:]
            train_spectrogram = new_data
            cols = cols + cols_to_add

        train_spectrogram = train_spectrogram.reshape(1, 1, rows, cols)

        PTS = cols // NUM_COLS

        for j in range(PTS):

            with open('./Spectrograms/spec' + str(index) + '.ds', 'wb') as fd:
                pickle.dump({'data' : train_spectrogram[:, :, 1 : 1025, j*NUM_COLS : (j+1)*NUM_COLS].float()}, fd)

            index = index+1




In [48]:
!rm -r ./Spectrograms/

In [49]:
!mkdir ./Spectrograms/

In [50]:
create_dataset(trainloader)

WORKING ON SPECTROGRAM NUMBER  0
WORKING ON SPECTROGRAM NUMBER  2
WORKING ON SPECTROGRAM NUMBER  4
WORKING ON SPECTROGRAM NUMBER  6
WORKING ON SPECTROGRAM NUMBER  8
WORKING ON SPECTROGRAM NUMBER  10
WORKING ON SPECTROGRAM NUMBER  12
WORKING ON SPECTROGRAM NUMBER  14
WORKING ON SPECTROGRAM NUMBER  16
WORKING ON SPECTROGRAM NUMBER  18
WORKING ON SPECTROGRAM NUMBER  20
WORKING ON SPECTROGRAM NUMBER  22
WORKING ON SPECTROGRAM NUMBER  24
WORKING ON SPECTROGRAM NUMBER  26
WORKING ON SPECTROGRAM NUMBER  28
WORKING ON SPECTROGRAM NUMBER  30
WORKING ON SPECTROGRAM NUMBER  32
WORKING ON SPECTROGRAM NUMBER  34
WORKING ON SPECTROGRAM NUMBER  36
WORKING ON SPECTROGRAM NUMBER  38
WORKING ON SPECTROGRAM NUMBER  40
WORKING ON SPECTROGRAM NUMBER  42
WORKING ON SPECTROGRAM NUMBER  44
WORKING ON SPECTROGRAM NUMBER  46
WORKING ON SPECTROGRAM NUMBER  48
WORKING ON SPECTROGRAM NUMBER  50
WORKING ON SPECTROGRAM NUMBER  52
WORKING ON SPECTROGRAM NUMBER  54
WORKING ON SPECTROGRAM NUMBER  56
WORKING ON SPECTROG

WORKING ON SPECTROGRAM NUMBER  472
WORKING ON SPECTROGRAM NUMBER  474
WORKING ON SPECTROGRAM NUMBER  476
WORKING ON SPECTROGRAM NUMBER  478
WORKING ON SPECTROGRAM NUMBER  480
WORKING ON SPECTROGRAM NUMBER  482
WORKING ON SPECTROGRAM NUMBER  484
WORKING ON SPECTROGRAM NUMBER  486
WORKING ON SPECTROGRAM NUMBER  488
WORKING ON SPECTROGRAM NUMBER  490
WORKING ON SPECTROGRAM NUMBER  492
WORKING ON SPECTROGRAM NUMBER  494
WORKING ON SPECTROGRAM NUMBER  496
WORKING ON SPECTROGRAM NUMBER  498
WORKING ON SPECTROGRAM NUMBER  500
WORKING ON SPECTROGRAM NUMBER  502
WORKING ON SPECTROGRAM NUMBER  504
WORKING ON SPECTROGRAM NUMBER  506
WORKING ON SPECTROGRAM NUMBER  508
WORKING ON SPECTROGRAM NUMBER  510
WORKING ON SPECTROGRAM NUMBER  512
WORKING ON SPECTROGRAM NUMBER  514
WORKING ON SPECTROGRAM NUMBER  516
WORKING ON SPECTROGRAM NUMBER  518
WORKING ON SPECTROGRAM NUMBER  520
WORKING ON SPECTROGRAM NUMBER  522
WORKING ON SPECTROGRAM NUMBER  524
WORKING ON SPECTROGRAM NUMBER  526
WORKING ON SPECTROGR

OSError: [Errno 28] No space left on device